# PDF Reader - Proposta EJEP

## Lista de funções

### Inicialização
- biblioteca: importa bibliotecas utilizadas no programa
- leitura_pdf: abre o pdf, pega as variaveis de diretorio, extrai texto
- xld

## Import bibliotecas

In [3]:
import PyPDF2
import datetime
import numpy as np
import pandas as pd
from os import walk
import os

In [4]:
import sys
import os
import comtypes.client

In [5]:
from os import chdir, getcwd, listdir, path
from win32com import client

In [6]:
import sys
import os
import os.path
import string
from textwrap import TextWrapper
import docx

In [7]:
import sys
from docx import Document

## Leitura do PDF

In [8]:
def leitura_docx(caminho):
    document = Document(caminho)
    texto = []
    for paragraph in document.paragraphs:
        texto.append(paragraph.text)

    # Juntar texto das páginas
    texto = ''.join(texto)
    texto = texto.lower()
    
    #Criar uma lista com nome dos diretorios e por último nome do arquivo
    caminho = str(caminho)
    caminho = caminho.split("\\")
    
      #Pegar nome importantes do diretorio
    if len(caminho) > 5:
        tipo_arq = caminho[4]
    else:
        tipo_arq = np.nan
    resultado = caminho[2]
    if caminho[3].find(" - ") != -1:
        apelido_area = caminho[3].split(" - ")
        apelido = apelido_area[0]
        area = apelido_area[1]
        nome_arq = caminho[-1]
    else:
        apelido = np.nan
        area = np.nan
        nome_arq = np.nan
    
    return texto, resultado, apelido, area, nome_arq, tipo_arq

In [9]:
def leitura_pdf(caminho):
    """
    Função que recebe o caminho do arquivo, abre o arquivo e entrega o texto.
    Como complemento, entrega variáveis com nomes das pastas (incompleto)
    
    INPUT:
    - caminho: string com endereço do arquivo
    
    OUTPUT:
    - texto: string com todo o texto do pdf 
    """
    #Abre pdf e faz uma lista de textos das páginas na variavel pdf
    f = open(caminho, 'rb')
    try:
        pdf = PyPDF2.PdfFileReader(f)
    except:
        texto = ''
        resultado = np.nan
        apelido = np.nan
        area = np.nan
        nome_arq = np.nan
        tipo_arq = np.nan
        return texto, resultado, apelido, area, nome_arq, tipo_arq
    
    #Importar módulo que trabalha com diretorios
    import os

    
    # Extratir texto página por página
    texto = []
    for i in range(pdf.getNumPages()):
        textopag = pdf.getPage(i).extractText()
        texto.append(textopag)
        
        
    # Juntar texto das páginas
    texto = ''.join(texto)
    texto = texto.lower()
    
    #Criar uma lista com nome dos diretorios e por último nome do arquivo
    caminho = str(caminho)
    caminho = caminho.split("\\")
    
      #Pegar nome importantes do diretorio
    if len(caminho) > 5:
        tipo_arq = caminho[4]
    else:
        tipo_arq = np.nan
    resultado = caminho[2]
    if caminho[3].find(" - ") != -1:
        apelido_area = caminho[3].split(" - ")
        apelido = apelido_area[0]
        area = apelido_area[1]
        nome_arq = caminho[-1]
    else:
        apelido = np.nan
        area = np.nan
        nome_arq = np.nan
        
    return texto, resultado, apelido, area, nome_arq, tipo_arq
    ## Consultar a ordem das pastas para associar a variaveis e pegar informaçoes do nome dos diretórios

In [10]:
def ver_leitura_pdf(texto):
    """ 
    Verifica se o comprimento do texto da leitura de pdf é maior que 500.
    
    INPUT:
    - texto: string com todo o texto do pdf 
    
    OUTPUT
    - ver_leitura_pdf: verificação da condição de validação da fução
    (=1 se estiver ok, =0 se for necessária revisão)
    
    """

    if len(texto) < 300:
        ver_leitura_pdf = 0
    else:
        if texto.find("capital de investimento") == -1 and texto.find("capital investido") == -1:
            ver_leitura_pdf = 0
        else:
            ver_leitura_pdf = 1
    return ver_leitura_pdf

### Extrair data do texto

In [11]:
def extrair_data(texto):
    
    """
    Extrai o valor da data em no texto e entrega como uma lsita de 3 valores
    
    INPUT:
    - texto: string com todo o texto do pdf 
    
    OUTPUT:
    - data: lista com 3 valores da data propsota [DD, MM, AAAA]
    
    """
    
    
    pos_cab = texto.find("florianópolis,")
    if pos_cab != -1:
        cabecalho = texto[pos_cab + 14:]
        n = ["\n", "."]
        count = 0
        for i in range(0,len(n)):
            valorlista = n[count]
            cabecalho = cabecalho.replace(valorlista,"")
            count += 1
        data = cabecalho.split()
        
        while ("de" in data) == True:
            data.remove("de")

    else:
        data = np.nan
    
    return data

Definindo biblioteca de meses

In [12]:
def convert_data(data):
    """
    Conversor de data do formato lista para datetime
    
    INPUT:
    - data: lista com 3 valores da data propsota [DD, MM, AAAA]
    
    OUTPUT:
    - dataf: data no formato datemtime

    """
    
    import datetime
    #Definindo biblioteca de meses
    meses = {
        "janeiro": 1,
        "fevereiro": 2,
        "março": 3,
        "abril": 4,
        "maio": 5,
        "junho": 6,
        "julho": 7,
        "agosto": 8,
        "setembro": 9,
        "outubro": 10,
        "novembro": 11,
        "dezembro": 12
    }
    
    # Convertendo para formato date
    pos_cab = texto.find("florianópolis,")
    if pos_cab != -1:
        try:
            dataf = datetime.date(int(data[2]), meses[data[1]], int(data[0]) )
        except ValueError:
            dataf = np.nan
        except KeyError:
            dataf = np.nan
    else:
        dataf = np.nan
        
    return dataf

### Investimento 

In [13]:
def extrair_invest(texto):
    """
    Extrai investimento e entrega em formato float.
    
    INPUT:
    - texto: string com todo o texto do pdf 
    
    OUTPUT:
    - valor_invest: valor da proposta em float.
    
    """

    pos_invest = texto.find("capital de investimento será de")
    if pos_invest != -1:
        texto_invest = texto[pos_invest + 33:]
        n = ["\n", "."]
        count = 0
        for i in range(0,len(n)):
            valorlista = n[count]
            texto_invest = texto_invest.replace(valorlista,"")
            count += 1
        valor_invest = texto_invest[texto_invest.find("$")+1:texto_invest.find("(") - 1]
        valor_invest = valor_invest.replace(",",".")
        try:
            valor_invest = float(valor_invest)
        except ValueError:
            valor_invest = np.nan
    else:
        texto_invest = np.nan
        valor_invest = np.nan
        
    return valor_invest

### Tempo de projeto

In [14]:
def extrair_tempo(texto):
    """
    Extrair tempo do projeto em semanas no formato float
    
    INPUT: 
    - texto: string com todo o texto do pdf 
    
    OUTPUT:
    - tempo: tempo do projeto em semanas no formato float
    """


    pos_invest = texto.find("capital de investimento será de")
    pos_cronograma = texto.find("conclusão será de")
    
    if pos_invest != -1 and pos_cronograma != -1:
        pos_tempo = texto.find("semanas", pos_cronograma)
    
        if pos_tempo != -1:
            texto_tempo = texto[pos_cronograma+18:pos_tempo-1]
            texto_tempo = texto_tempo.split()
            texto_tempo = ''.join(texto_tempo)
            try:
                tempo = float(texto_tempo)
            except ValueError:
                tempo = np.nan
        else:
             tempo = np.nan
    else:
        tempo = np.nan
        
    return tempo

### Área de atuação

Função que identifica se a área está ou não no texto e retorna um binário. 

In [15]:
def id_find(valor, texto):
    """
    Função que identifica se a área está ou não no texto e retorna um binário. 
    
    INPUT:
    - valor: valor a ser procurado no texto
    - texto: string com todo o texto do pdf 
    
    OUTPUT: 
    - binário id_find: 1 se achou no texto, 0 se nao ahcou.
    
    """
    if texto.find(valor) != -1:
        return 1
    else:
        return 0

Aplicando a função nas áreas de atuação.

In [16]:
def extrair_area(texto):
    """
    Função que atribui binário às variáveis de área de atuação se está presente no texto ou não.
    
    INPUT: 
    - texto: string com todo o texto do pdf
    
    OUTPUT:
    Binário de cada área de atução
    - area_trf
    - area_pcp
    - area_sag
    - area_logistica
    - area_mapeamento
    - area_mkt
    - area_5s
    - area_layout
    - area_mfv
    - area_outro
    """

    area_trf = id_find("trf",texto)
    area_pcp = id_find("pcp",texto)
    area_sag = id_find("sag", texto)
    area_logistica = id_find("logística", texto)
    area_mapeamento = id_find("mapeamento de processos", texto)
    area_mkt = id_find("plano de marketing", texto)
    area_pe = id_find("planejamento estratégico", texto)
    area_5s = id_find("5s", texto)
    area_layout = id_find("layout", texto)
    area_mfv = id_find("mfv", texto)
    area_outro = 0 
    
    return area_trf, area_pcp, area_sag, area_logistica, area_mapeamento, area_mkt, area_pe, area_5s, area_layout, area_mfv, area_outro

Revisor de coerência do valor encontrado

In [17]:
def ver_extrair_area(vetor_area):
    """
    Verificador se área de atuação está coerente. Analisa o número de áreas encontradas para tomada de decisão manual. 
    Além disso se nenhuma área de atuação for encontrada atribui valor 1 para outro. 
    
    INPUT:
    - vetor_area: vetor com valores de binários de área de atuação.
    
    OUTPU:
    - revisor: análise dos valores de área de atuação.
        Se nenhuma área for encontrada: "Nenhuma área encontrada"
        Se uma área for encontrada: "Uma área encontrada"
        Se mais de uma área encontrada: "Mais de uma área encontrada"
        
    OBS: rever essas labels
    """

    soma = 0
    for area in vetor_area:
        soma += area
        
    if soma > 1:
        revisor = "Mais de uma área encontrada"
        
    else:
        if soma == 1:
            revisor = "Uma área encontrada"
            
        else:
            revisor = "Nenhuma área encontrada"
            area_outro = 1
            
    return revisor

Dicionário apenas para teste, não incluir no código!

### Entidades participantes

In [18]:
def extrair_entidades(texto):
    """
    Identifica se existe a palavra "glean" ou "glog" no texto.
    
    INPUT:
    - texto: string com todo o texto do pdf
    
    OUTPUT:
    - glean: binário se a palavra "glean" está presente no texto
    - glog: binário se a palavra "glog" está presente no texto
    """

    ent_glean = id_find("glean", texto)
    ent_glog = id_find("glog", texto)
    
    return ent_glean, ent_glog

### Report de erros e pontos de melhoria

Área de atuação:
- Ás vezes nas áreas de lean cita uma mini etapa de 5S, mas não sei se deve considerar
    - Possível solução: sempre que tiver outra área, 5S = 0
- Na área de PCP pode haver uma etapa de Planejamento Estratégico da Produção, o que faz com que o valor de area_pe vá para 1

Leitura do arquivo:
- Qunado se faz o download de um arquivo de pdf de um google doc o código não funciona. É necessário fazer o download em word e converter para PDF

# Replicando para todos os arquivos

## Listar arquivos na pasta

Função de walk de 1 nível

In [19]:
def walk_file_folder(arquivos,pastas):
    """
    Função que varre 1 nível de arquivos e pastas
    
    INPUT
    - arquivos: lista de arquivos já mapeados
    - pastas: pastas a serem varrias
    
    OUPUT
    - arquivos: lista de arquivos já mapeados 
    - pastasn: pastas varridas no nível analisado
    
    """
    pastasn = []    
    for pasta in pastas:
    # Ler arquivos e pastas do diretório atual
        for (dirpath, dirnames, filenames) in walk(pasta):
            break

    #adicionar aquivos a lista de arquivos a ser lidos
        for name in filenames:
            arquivos.append(os.path.join(dirpath, name))

    
    #criar lista de pastas a ser walkiadas    
        for pastan in dirnames:
            pastasn.append(os.path.join(dirpath, pastan))

    return arquivos, pastasn

Iterações de varredura de arquivos

In [20]:
def walk_todas_pastas():
    """
    Função de iteração da funçao walk_file_folder. Roda a função em todos os níveis de pastas.
    
    INPUT: não possui
    
    OUTPUT: 
    - arquivos: lista de arquivos já mapeados 
    """

    pastas = ['.']
    arquivos = []
    count = 1

    while pastas != []:
        pastas = walk_file_folder(arquivos,pastas)[1]
        
    return arquivos




### Limpar arquivos que não são PDF


In [21]:
def drop_pdf_doc(arquivos):
    """
    Função que elimina arquios que não são pdf.
    
    INPUT:
    - arquivos: arquivos mapeados
    
    OUTPUT:
    - arquivos_pdf: arquivos mapeados só considerando pdf
    """ 
    arquivo_pdf = []

    for arq in arquivos:
        if arq.endswith('pdf') or arq.endswith('doc') or arq.endswith('docx'):
            arquivo_pdf.append(arq)
            
    return arquivo_pdf

## Agregar dados ao dataframe

### Criando o dataframe
obs: falta agregar os dados dos nomes do diretorio que depende de me colocarem na pasta do EJEPSERVER

In [22]:
def criar_dataframe(colunas):
    """
    Função de criação de dataframe
    
    INPUT:
    - colunas: lista de nome das colunas
    
    OUTPUT
    - df: dataframe sem dados com "colunas" como labels de coluna
    """
    df = pd.DataFrame(columns = colunas)
    return df

### Acrescentando dados da proposta analisada

In [23]:
def add_dados(df, variaveis,colunas):
    """
    Adiciona dados a um dataframe
    
    INPUT:
    - df: dataframe a ser adicionado os dodos
    - variaveis: dados a serem adicionados
    - colunas: colunas do dataframe
    
    OUTPUT:
    - df: dataframe com dados adicionados
    
    """
    df2 = pd.DataFrame(data = [variaveis], columns = colunas)
    df = df.append(df2)
    
    return df

# Script

### Import bibliotecas

In [24]:
import PyPDF2
import datetime
import numpy as np
import pandas as pd
from os import walk
import os
import sys
from docx import Document
import docx
import time

### Criar vetor caminho

In [25]:
start_time = time.time()

arquivos = walk_todas_pastas()

print('Análise antes de drop_pdf:')
#print(arquivos)
print(len(arquivos))

arquivos_pdf_doc = drop_pdf_doc(arquivos)

print('Análise depois de drop_pdf')
#print(arquivos_pdf_doc)

print(len(arquivos_pdf_doc))
print(arquivos_pdf_doc)

print("--- %s seconds ---" % (time.time() - start_time))

Análise antes de drop_pdf:
246
Análise depois de drop_pdf
87
['.\\2018\\Em processo de venda\\Apetência T - Pesquisa de Mercado\\Apresentação de Proposta Apetência - Pesquisa de Mercado.docx', '.\\2018\\Em processo de venda\\Arzinho - MFV\\CAPA DA PROPOSTA - Fluair.docx', '.\\2018\\Em processo de venda\\Arzinho - MFV\\Proposta Arzinho.docx', '.\\2018\\Em processo de venda\\Breaking Bad - MP com Melhoria\\Proposta - Breaking Bad.docx', '.\\2018\\Em processo de venda\\Cheirosa - MFV\\Proposta Ciclo Cosmeticos.docx', '.\\2018\\Em processo de venda\\Cofrinho - MFV + Kanban\\Proposta Cofrinho.docx', '.\\2018\\Em processo de venda\\Creche - Gestão da Rotina\\Creche - Gestão da Rotina.docx', '.\\2018\\Em processo de venda\\Descubra - Plano de Negócios\\Proposta Descubra - Análise Finaceira + Custos Logísticos + Estudo de Maquinário.docx', '.\\2018\\Em processo de venda\\Hospital II - Análise de Mercado\\Proposta Hospital II.docx', '.\\2018\\Em processo de venda\\Kidzone - MFI\\Capa Kid .docx'

### Script final

In [26]:
start_time = time.time()
colunas = ["nome_arq", "tipo_arq", "revisor_texto", "data", "investimento", "tempo", 'area_trf', 'area_pcp', 'area_sag', 'area_logistica', 'area_mapeamento', 'area_mkt', 'area_pe','area_5s', 'area_layout', 'area_mfv', 'area_outro', 'revisor_area', 'ent_glean', 'ent_glog', 'resultado', 'apelido', 'area']

df = criar_dataframe(colunas)


for caminho in arquivos_pdf_doc:
    print(caminho)
    if caminho.endswith('pdf'):
        leitura = leitura_pdf(caminho)
    else:
        leitura = leitura_docx(caminho)
    if leitura != []:
        texto = leitura[0]
        resultado = leitura[1]
        apelido = leitura[2]
        area = leitura[3]
        nome_arq = leitura[4]
        tipo_arq = leitura[5]
    else:
        texto = np.nan
        resultado = np.nan
        apelido = np.nan
        area = np.nan
        nome_arq = np.nan
        tipo_arq = np.nan
    revisor_texto = ver_leitura_pdf(texto)
    data = convert_data(extrair_data(texto))
    valor_invest = extrair_invest(texto)
    tempo = extrair_tempo(texto)
    area_list = extrair_area(texto)
    revisor_area = ver_extrair_area(area_list)
    entidades = extrair_entidades(texto)
    variaveis = [nome_arq, tipo_arq, revisor_texto, data, valor_invest, tempo, area_list[0], area_list[1], area_list[2], area_list[3], area_list[4], area_list[5], area_list[6], area_list[7], area_list[8], area_list[9], area_list[10], revisor_area, entidades[0], entidades[1], resultado, apelido, area]
    df = add_dados(df, variaveis, colunas)

print("--- %s seconds ---" % (time.time() - start_time))

.\2018\Em processo de venda\Apetência T - Pesquisa de Mercado\Apresentação de Proposta Apetência - Pesquisa de Mercado.docx
.\2018\Em processo de venda\Arzinho - MFV\CAPA DA PROPOSTA - Fluair.docx
.\2018\Em processo de venda\Arzinho - MFV\Proposta Arzinho.docx
.\2018\Em processo de venda\Breaking Bad - MP com Melhoria\Proposta - Breaking Bad.docx
.\2018\Em processo de venda\Cheirosa - MFV\Proposta Ciclo Cosmeticos.docx
.\2018\Em processo de venda\Cofrinho - MFV + Kanban\Proposta Cofrinho.docx
.\2018\Em processo de venda\Creche - Gestão da Rotina\Creche - Gestão da Rotina.docx
.\2018\Em processo de venda\Descubra - Plano de Negócios\Proposta Descubra - Análise Finaceira + Custos Logísticos + Estudo de Maquinário.docx
.\2018\Em processo de venda\Hospital II - Análise de Mercado\Proposta Hospital II.docx
.\2018\Em processo de venda\Kidzone - MFI\Capa Kid .docx
.\2018\Em processo de venda\Kidzone - MFI\Kid.pdf
.\2018\Em processo de venda\Kidzone - MFI\Proposta KidZone.docx
.\2018\Em proces

In [27]:
df

,nome_arq,tipo_arq,revisor_texto,data,investimento,tempo,area_trf,area_pcp,area_sag,area_logistica,...,area_5s,area_layout,area_mfv,area_outro,revisor_area,ent_glean,ent_glog,resultado,apelido,area
0,Apresentação de Proposta Apetência - Pesquisa ...,NaN,1,NaN,NaN,7.0,0,0,0,0,...,0,0,0,0,Uma área encontrada,0,0,Em processo de venda,Apetência T,Pesquisa de Mercado
0,CAPA DA PROPOSTA - Fluair.docx,NaN,0,NaN,NaN,NaN,0,0,0,1,...,0,0,0,0,Uma área encontrada,0,0,Em processo de venda,Arzinho,MFV
0,Proposta Arzinho.docx,NaN,1,2018-04-09,9725.0,11.0,0,0,0,1,...,0,1,1,0,Mais de uma área encontrada,0,0,Em processo de venda,Arzinho,MFV
0,Proposta - Breaking Bad.docx,NaN,1,2017-09-15,12600.0,13.0,0,0,0,0,...,0,0,0,0,Uma área encontrada,0,0,Em processo de venda,Breaking Bad,MP com Melhoria
0,Proposta Ciclo Cosmeticos.docx,NaN,1,2018-03-28,NaN,NaN,0,0,1,0,...,0,0,1,0,Mais de uma área encontrada,0,0,Em processo de venda,Cheirosa,MFV
0,Proposta Cofrinho.docx,NaN,1,2018-03-23,10200.0,11.0,0,0,0,0,...,0,1,1,0,Mais de uma área encontrada,0,0,Em processo de venda,Cofrinho,MFV + Kanban
0,Creche - Gestão da Rotina.docx,NaN,1,2018-02-28,9300.0,14.0,0,0,0,0,...,0,0,0,0,Nenhuma área encontrada,0,0,Em processo de venda,Creche,Gestão da Rotina
0,Proposta Descubra - Análise Finaceira + Custos...,NaN,1,2018-04-20,NaN,NaN,0,0,0,1,...,0,0,0,0,Uma área encontrada,0,0,Em processo de venda,Descubra,Plano de Negócios
0,Proposta Hospital II.docx,NaN,1,2018-04-17,6150.0,10.0,0,0,0,0,...,0,0,0,0,Nenhuma área encontrada,0,0,Em processo de venda,Hospital II,Análise de Mercado
0,Capa Kid .docx,NaN,0,NaN,NaN,NaN,0,0,0,0,...,0,0,0,0,Nenhuma área encontrada,0,0,Em processo de venda,Kidzone,MFI


In [29]:
df[df["revisor_texto"] == 1]

,nome_arq,tipo_arq,revisor_texto,data,investimento,tempo,area_trf,area_pcp,area_sag,area_logistica,...,area_5s,area_layout,area_mfv,area_outro,revisor_area,ent_glean,ent_glog,resultado,apelido,area
0,Apresentação de Proposta Apetência - Pesquisa ...,NaN,1,NaN,NaN,7.0,0,0,0,0,...,0,0,0,0,Uma área encontrada,0,0,Em processo de venda,Apetência T,Pesquisa de Mercado
0,Proposta Arzinho.docx,NaN,1,2018-04-09,9725.0,11.0,0,0,0,1,...,0,1,1,0,Mais de uma área encontrada,0,0,Em processo de venda,Arzinho,MFV
0,Proposta - Breaking Bad.docx,NaN,1,2017-09-15,12600.0,13.0,0,0,0,0,...,0,0,0,0,Uma área encontrada,0,0,Em processo de venda,Breaking Bad,MP com Melhoria
0,Proposta Ciclo Cosmeticos.docx,NaN,1,2018-03-28,NaN,NaN,0,0,1,0,...,0,0,1,0,Mais de uma área encontrada,0,0,Em processo de venda,Cheirosa,MFV
0,Proposta Cofrinho.docx,NaN,1,2018-03-23,10200.0,11.0,0,0,0,0,...,0,1,1,0,Mais de uma área encontrada,0,0,Em processo de venda,Cofrinho,MFV + Kanban
0,Creche - Gestão da Rotina.docx,NaN,1,2018-02-28,9300.0,14.0,0,0,0,0,...,0,0,0,0,Nenhuma área encontrada,0,0,Em processo de venda,Creche,Gestão da Rotina
0,Proposta Descubra - Análise Finaceira + Custos...,NaN,1,2018-04-20,NaN,NaN,0,0,0,1,...,0,0,0,0,Uma área encontrada,0,0,Em processo de venda,Descubra,Plano de Negócios
0,Proposta Hospital II.docx,NaN,1,2018-04-17,6150.0,10.0,0,0,0,0,...,0,0,0,0,Nenhuma área encontrada,0,0,Em processo de venda,Hospital II,Análise de Mercado
0,Proposta KidZone.docx,NaN,1,2018-04-19,NaN,NaN,0,0,0,0,...,0,0,0,0,Nenhuma área encontrada,0,0,Em processo de venda,Kidzone,MFI
0,Proposta Lanoposto.docx,NaN,1,2018-03-16,NaN,4.0,0,0,0,0,...,0,0,0,0,Nenhuma área encontrada,0,0,Em processo de venda,Lanoposto,Fluxo caixa


In [31]:
df[df["nome_arq"] =='Proposta Costela.docx']

,nome_arq,tipo_arq,revisor_texto,data,investimento,tempo,area_trf,area_pcp,area_sag,area_logistica,...,area_5s,area_layout,area_mfv,area_outro,revisor_area,ent_glean,ent_glog,resultado,apelido,area
0,Proposta Costela.docx,Venda,1,2018-04-04,8500.0,12.0,0,0,0,0,...,0,0,0,0,Nenhuma área encontrada,0,0,Vendidos,Costela,Gestão Rotina
